2025/04/30 動くか試しに実行
動いた！
５交差検証法を用いた実装。

#References: https://qiita.com/tetsuro731/items/671c81669955d98bb6ff
#概要：LightGBM実行して、テスト結果をMLFlowで確認しよう！

2025/11/16
■運用手順
①Gogle Colabにてmlflowサーバを起動
②起動したURLをmlflow.set_tracking_uri　にセットする
③プログラムを実行する

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
cd drive/MyDrive/20230614_新機械学習/

/content/drive/MyDrive/20230614_新機械学習


In [ ]:
!pip install lightgbm
!pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 108.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.8/76.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.9/753.9 kB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 22.1 MB/s eta 0:00:00


In [ ]:
import seaborn as sns
data = sns.load_dataset('titanic')


In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import mlflow


# Encode categorical features
le = LabelEncoder()
for col in ['sex', 'embarked', 'who', 'embark_town', 'alive']:
    data[col] = le.fit_transform(data[col])

# 特徴量とターゲットに分割
N_splits = 5
train = data.drop('survived', axis=1)
y = data['survived']

#mlflow.lightgbm.autolog() #mlfllow実行用
mlflow.set_tracking_uri("https://9175661e3837.ngrok-free.app")#URI発行したらメンテする必要ある。
mlflow.set_experiment("colab-demo")#demo名かえるならメンテ

kfold = KFold(n_splits=N_splits, shuffle=True, random_state=42)
for fold, (trn_ind, val_ind) in enumerate(kfold.split(train)):
  with mlflow.start_run():
    print(' ')
    print('-'*50)
    print(f'Training fold {fold}/{N_splits}....')
    x_train, x_val = train.iloc[trn_ind], train.iloc[val_ind]
    y_train, y_val = y[trn_ind], y[val_ind]

    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'num_leaves': 31,
        'learning_rate': 0.1,
        'feature_fraction': 0.9,
        'verbosity': -1,
    }
    verbose_eval = 20
    evals_result = {}

    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_valid = lgb.Dataset(x_val, y_val, reference=lgb_train)
    model = lgb.train(
      params = params,
      train_set = lgb_train,
      num_boost_round = 100,
      valid_sets=[lgb_train, lgb_valid],
      callbacks=[lgb.early_stopping(stopping_rounds=20,  verbose=True),
                 lgb.log_evaluation(verbose_eval),
                 lgb.record_evaluation(evals_result)]
    )

    y_pred = model.predict(x_val, num_iteration=model.best_iteration)
    accuracy = accuracy_score(y_val, np.round(y_pred))
    print(f'Fold {fold}, Accuracy: {accuracy}')
    # ※X_trainから1行を抽出
    input_example = pd.DataFrame(x_train.iloc[0]).T  # 最初の1行を入力例として指定

    # モデルをMLflowに保存
    # mlflow.lightgbm.log_model(model, "lightgbm_model", input_example=input_example)

    # パラメータをMLflowに記録
    for param, value in params.items():
        mlflow.log_param(param, value)

    # メトリクスをMLflowに記録
    mlflow.log_metric("best_iteration", model.best_iteration)
    mlflow.log_metric("train_error", evals_result['training']['binary_logloss'][model.best_iteration-1]) # Changed 'train' to 'training' and used best_iteration-1 for index
    mlflow.log_metric("val_error", evals_result['valid_1']['binary_logloss'][model.best_iteration-1]) # Used best_iteration-1 for index

 
--------------------------------------------------
Training fold 0/5....
Training until validation scores don't improve for 20 rounds
[20]	training's binary_logloss: 0.0634963	valid_1's binary_logloss: 0.0647114
[40]	training's binary_logloss: 0.00832195	valid_1's binary_logloss: 0.00847608
[60]	training's binary_logloss: 0.00124187	valid_1's binary_logloss: 0.00127745
[80]	training's binary_logloss: 0.000188665	valid_1's binary_logloss: 0.000196309
[100]	training's binary_logloss: 2.9012e-05	valid_1's binary_logloss: 3.05808e-05
Did not meet early stopping. Best iteration is:
[100]	training's binary_logloss: 2.9012e-05	valid_1's binary_logloss: 3.05808e-05
Fold 0, Accuracy: 1.0
🏃 View run colorful-ant-560 at: https://9175661e3837.ngrok-free.app/#/experiments/1/runs/689ce04a3af14fa29297b728dd0a8582
🧪 View experiment at: https://9175661e3837.ngrok-free.app/#/experiments/1
 
--------------------------------------------------
Training fold 1/5....
Training until validation scores don't 

In [ ]:
accuracy_score(y_val, np.round(y_pred))

1.0